## Run UDFs on Merlin live streams - minimal example

This is a minimal example that shows how to run LiberTEM user-defined functions (UDFs) on Merlin Medipix live data streams. This example assumes that triggering is set to soft trigger, and all other detector settings are already set to the correct values.

* Make sure to adjust the `nav_shape` parameter below to match the scan of the data source!
* This notebook requires the `bqplot` extra of LiberTEM: `pip install libertem[bqplot]`

### Usage with the simulator

If you want to use this with the simulated data source, run a simple Merlin simulator in the background that replays an MIB dataset:

`libertem-live-mib-sim ~/Data/default.hdr --cached=MEM --wait-trigger`

The `--wait-trigger` option is important for this notebook to function correctly since that allows to drain the data socket before an acquisition like it is necessary for a real-world Merlin detector.

A suitable MIB dataset can be downloaded at https://zenodo.org/record/5113449.

On Linux, `MEMFD` is also supported as a cache. Use `NONE` to deactivate the cache.

## Parameters

In [1]:
# The shape of your scan, we assume a 2D scan here. adjust to match microscope settings:
NAV_SHAPE = (128, 128)

# Change this if you are running on a different host than the PC connected directly to the detector:
HOST = '127.0.0.1'

## General setup

In [2]:
from libertem_live.api import LiveContext, Hooks
from libertem.viz.bqp import BQLive2DPlot

In [3]:
ctx = LiveContext(
    plot_class=BQLive2DPlot,  # use webgl based plotting for more efficient plot updates
)

## Connection to the detector software

Here, we connect to the detector software for the Merlin Medipix detector. This immediately establishes a connection, so that we get an early error. If you are using the `with conn: ...` construct, as shown below, this connection will be closed at the end of each acquisition.

In [4]:
api_host = data_host = HOST

conn = ctx.make_connection('merlin').open(
    data_host=data_host,
    data_port=6342,
    api_host=api_host,
    api_port=6341,
)

## What computation do we want to run on the live stream?

In [5]:
# Sum all detector frames, result is a map of the detector
from libertem.udf.sum import SumUDF

# Sum up each detector frame, result is a bright field STEM image of the scan area
from libertem.udf.sumsigudf import SumSigUDF

# Sample individual detector frames from the live stream
from libertem_live.udf.monitor import SignalMonitorUDF

In [6]:
udfs = [SumUDF(), SumSigUDF(), SignalMonitorUDF()]

### Integration Hooks

A LiberTEM Live acquisition object can include a hooks object, so that LiberTEM Live can set off the acquisition as soon as it has connected to the camera and is ready to receive data. The `on_ready_for_data` function receives an environment as argument, from which you can access the current acquisition object as the attribute `aq`.

In [7]:
class MerlinHooks(Hooks):
    def __init__(self):
        self.trigger_result = None
        
    def on_ready_for_data(self, env):
        print(f"Arming and triggering Merlin for a scan of {env.aq.shape.nav}...")
        with conn.control() as c:
            # Arm the detector and sends the acquisition headers:
            c.cmd('STARTACQUISITION')
            c.cmd('SOFTTRIGGER')

In [8]:
hooks = MerlinHooks()
aq = ctx.make_acquisition(
    conn=conn,
    hooks=hooks,
    nav_shape=NAV_SHAPE,

    frames_per_partition=800,
)

### Run one scan

The live plots above are updated with the results

In [9]:
# Using `with conn` to clean up the data connection after we are finished with the acquisition.
# Afterwards, you can do the same again and a new connection will automatically be established.
with conn:
    
    # This will call the `on_ready_for_data` hook defined above as soon as
    # LiberTEM-live is ready to receive data.
    ctx.run_udf(dataset=aq, udf=udfs, plots=True)
    print("Finished.")

Arming and triggering Merlin for a scan of (128, 128)...


Figure(axes=[Axis(label='x', scale=LinearScale(max=1.0, min=0.0)), Axis(label='y', orientation='vertical', sca…

Figure(axes=[Axis(label='x', scale=LinearScale(max=1.0, min=0.0)), Axis(label='y', orientation='vertical', sca…

Figure(axes=[Axis(label='x', scale=LinearScale(max=1.0, min=0.0)), Axis(label='y', orientation='vertical', sca…

Finished.
